# Vector Search with BrickKit

This notebook demonstrates **end-to-end usage of BrickKit** for deploying a governed Vector Search solution.

## What BrickKit Does

BrickKit automates governance for Databricks resources:
- **Team management** - Define teams with workspaces and principals, auto-configure catalog bindings
- **Principal management** - Define groups and service principals with environment-aware naming
- **Naming conventions** - Environment-aware names (dev/acc/prd suffixes)
- **Tagging** - Automatic cost center, team, compliance tags
- **Ownership rules** - Enforce service principals for catalogs, groups for schemas
- **Permission grants** - Ensure teams retain access after ownership changes
- **Request for Access (RFA)** - Configure access request destinations with inheritance
- **Validation** - Catch governance violations before deployment

## BrickKit vs DAB (Databricks Asset Bundles)

| Resource | DAB | BrickKit | Notes |
|----------|-----|----------|-------|
| **Teams** | Not supported | Defines & manages | BrickKit organizes workspace + principals |
| **Groups/SPNs** | Not supported | Defines & manages | BrickKit defines principals declaratively |
| **Catalog** | References only | Creates & governs | DAB passes variables, BrickKit deploys |
| **Schema** | References only | Creates & governs | Same |
| **Table** | References only | Creates & governs | BrickKit defines structure + tags |
| **VS Endpoint** | Not supported | Creates & governs | DAB can't create these |
| **VS Index** | Not supported | Creates & governs | DAB can't create these |
| **Jobs/Workflows** | Defines | N/A | DAB's strength |
| **Notebook sync** | Deploys | N/A | DAB syncs to workspace |

**Key insight:** DAB deploys *code assets* (notebooks, jobs). BrickKit deploys *data assets* (catalogs, tables, VS) and *principals* (teams, groups, SPNs).

## What This Demo Shows

1. Load a governance convention from YAML
2. **Define Team** with workspace and principals
3. Define governed resources (Catalog, Schema, Table, VS Endpoint, VS Index)
4. **Auto-configure workspace bindings** via `team.add_catalog()`
5. Deploy using BrickKit executors
6. **Grant permissions** to team groups after ownership change
7. Test vector search
8. See what governance BrickKit applied automatically

---
## 1. Configuration

In [ ]:
%pip install --upgrade databricks-vectorsearch databricks-sdk pydantic pyyaml --quiet
dbutils.library.restartPython()

In [ ]:
# === CONFIGURATION ===
# Edit these widgets or override via DAB job parameters

dbutils.widgets.text("catalog", "quant_risk", "Catalog Name (base)")
dbutils.widgets.text("schema", "indicators", "Schema Name")
dbutils.widgets.text("endpoint_name", "worldbank_vector_search", "VS Endpoint Name")
dbutils.widgets.text("managed_location", "", "Managed Location (for Default Storage workspaces)")
dbutils.widgets.dropdown("environment", "dev", ["dev", "acc", "prd"], "Environment")
dbutils.widgets.dropdown("dry_run", "false", ["true", "false"], "Dry Run")

# Read widget values
CATALOG_BASE = dbutils.widgets.get("catalog")
SCHEMA_NAME = dbutils.widgets.get("schema")
ENDPOINT_NAME = dbutils.widgets.get("endpoint_name")
MANAGED_LOCATION = dbutils.widgets.get("managed_location") or None  # Convert empty string to None
ENVIRONMENT = dbutils.widgets.get("environment")
DRY_RUN = dbutils.widgets.get("dry_run").lower() == "true"

# Derived names
TABLE_NAME = "worldbank_indicators"
INDEX_NAME = f"{TABLE_NAME}_index"

print(f"Environment: {ENVIRONMENT}")
print(f"Dry Run: {DRY_RUN}")
print(f"Catalog (base): {CATALOG_BASE}")
print(f"Schema: {SCHEMA_NAME}")
print(f"Endpoint: {ENDPOINT_NAME}")
if MANAGED_LOCATION:
    print(f"Managed Location: {MANAGED_LOCATION}")

In [ ]:
# === IMPORTS ===
import logging
import sys
import os
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.utils import AnalysisException

# Add brickkit to path (not yet published to PyPI)
# notebookPath() returns workspace-relative path, need /Workspace prefix for filesystem
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_dir = os.path.dirname(notebook_path)
# Prepend /Workspace for filesystem access
workspace_notebook_dir = f"/Workspace{notebook_dir}"
src_path = os.path.abspath(os.path.join(workspace_notebook_dir, "..", "..", "src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print(f"Added to sys.path: {src_path}")

from databricks.sdk import WorkspaceClient
from databricks.vector_search.client import VectorSearchClient

# BrickKit imports
from brickkit import (
    Catalog,
    Schema,
    Tag,
    SecurableType,
    VectorSearchEndpoint,
    VectorSearchIndex,
    load_convention,
)
from brickkit.models.tables import Table, ColumnInfo
from brickkit.models.grants import Principal, AccessPolicy
from brickkit.models.principals import ManagedGroup, ManagedServicePrincipal
from brickkit.models.enums import PrincipalType, IsolationMode
from brickkit.models.workspace_bindings import Workspace, WorkspaceRegistry
from brickkit.models.teams import Team
from brickkit.executors import (
    CatalogExecutor,
    SchemaExecutor,
    GrantExecutor,
    VectorSearchEndpointExecutor,
    VectorSearchIndexExecutor,
    ServicePrincipalExecutor,
    get_privileged_client,
)
from brickkit.models.base import set_current_environment
from brickkit.models.enums import Environment

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

# Set BrickKit environment
ENV_MAP = {"dev": Environment.DEV, "acc": Environment.ACC, "prd": Environment.PRD}
set_current_environment(ENV_MAP[ENVIRONMENT])

print(f"BrickKit environment set to: {ENVIRONMENT}")

In [ ]:
# === LOAD GOVERNANCE CONVENTION ===
# The convention defines naming patterns, required tags, and ownership rules

# Use absolute path based on notebook location
CONVENTION_PATH = os.path.join(workspace_notebook_dir, "conventions", "financial_services.yml")
convention = load_convention(CONVENTION_PATH)

print(f"Loaded convention: {convention.name} (v{convention.version})")
print(f"Rules: {len(convention.schema.rules)}")
print(f"Default tags: {len(convention.schema.tags)}")

# Show what the convention enforces
for rule in convention.schema.rules:
    mode = "ENFORCED" if rule.mode.value == "enforced" else "ADVISORY"
    print(f"  [{mode}] {rule.rule}")

### Bootstrap: Admin Service Principal

Notebook tokens have limited permissions (e.g., cannot update catalog isolation mode).
We create an "admin" service principal with OAuth credentials stored in Databricks Secrets.

**Bootstrap flow:**
1. Create admin SPN using your user token
2. Store SPN credentials in Databricks Secrets
3. Grant the admin SPN `MANAGE` + `USE_CATALOG` on the catalog (using your user token)
4. Create privileged client from stored credentials
5. Use privileged client for catalog operations

**First run:** Credentials are created and stored. You need to run the grant cell after defining the catalog.
**Subsequent runs:** Credentials loaded from secrets, privileged client ready to use.

In [ ]:
# === BOOTSTRAP: CREATE ADMIN SERVICE PRINCIPAL ===
# Creates an admin SPN with OAuth credentials for privileged operations.
# The SPN will be granted catalog access later (after catalog is defined).

ADMIN_SPN_NAME = "spn_brickkit_admin"
SECRET_SCOPE = "brickkit"

# Define the admin SPN using BrickKit's ManagedServicePrincipal
# Note: display_name defaults to resolved_name (e.g., spn_brickkit_admin_dev)
admin_spn = ManagedServicePrincipal(name=ADMIN_SPN_NAME)
admin_spn.add_entitlement("workspace-access")
admin_spn.add_entitlement("databricks-sql-access")

# Create a Principal reference for granting permissions
admin_spn_principal = Principal(
    name=ADMIN_SPN_NAME,
    principal_type=PrincipalType.SERVICE_PRINCIPAL,
    add_environment_suffix=True,
)

# Use default client (your user token) for bootstrap operations
bootstrap_client = WorkspaceClient()
spn_executor = ServicePrincipalExecutor(bootstrap_client, dry_run=DRY_RUN)

# Check if credentials exist in secrets AND the SPN actually exists in Databricks
CREDENTIALS_EXIST = False
ADMIN_SPN_APP_ID = None
SPN_EXISTS = False

# First check if SPN exists in Databricks
try:
    SPN_EXISTS = spn_executor.exists(admin_spn)
    if SPN_EXISTS:
        print(f"✓ SPN {admin_spn.resolved_name} exists in Databricks")
except Exception as e:
    print(f"Could not check SPN existence: {e}")

# Then check if credentials exist in secrets
try:
    ADMIN_SPN_APP_ID = dbutils.secrets.get(scope=SECRET_SCOPE, key="admin-spn-client-id")
    print(f"✓ Credentials found in scope '{SECRET_SCOPE}' (Application ID: {ADMIN_SPN_APP_ID})")
    
    # Only trust credentials if the SPN actually exists
    if SPN_EXISTS:
        CREDENTIALS_EXIST = True
    else:
        print(f"⚠ Credentials exist but SPN does not - will recreate SPN and update credentials")
except Exception:
    print(f"No credentials found in scope '{SECRET_SCOPE}'")

# Create SPN and store credentials if needed
if not CREDENTIALS_EXIST and not DRY_RUN:
    print(f"Creating admin SPN: {admin_spn.resolved_name}")
    result, credentials = spn_executor.create_with_secret(admin_spn)
    print(f"  {result.operation.value}: {result.message}")
    
    if credentials:
        # Store credentials in Databricks Secrets (overwrites if they exist)
        spn_executor.store_credentials(credentials, scope=SECRET_SCOPE)
        print(f"  ✓ Stored credentials in scope '{SECRET_SCOPE}'")
        ADMIN_SPN_APP_ID = credentials.application_id
        print(f"  Application ID: {ADMIN_SPN_APP_ID}")
        CREDENTIALS_EXIST = True
    elif result.operation.value == "NO_OP" and SPN_EXISTS:
        # SPN already exists but we don't have fresh credentials
        # This shouldn't happen with the logic above, but handle it
        print(f"  ⚠ SPN exists but no new credentials generated")
        CREDENTIALS_EXIST = False
elif DRY_RUN:
    print(f"[DRY RUN] Would create admin SPN: {admin_spn.resolved_name}")

print()
if CREDENTIALS_EXIST:
    print("✓ Admin SPN ready with valid credentials")
else:
    print("⚠ Admin SPN credentials not available - privileged operations may fail")

### Define Team, Workspace & Principals

BrickKit uses `Team` to bring together:
- **Workspace** - The Databricks workspace(s) per environment
- **Principals** - Service principals and groups that own/access resources
- **Catalog bindings** - Automatically configured via `team.add_catalog()`

In [ ]:
# === DEFINE WORKSPACE ===
# Register the workspace. In Free Edition with "Default Storage", catalogs are workspace-bound.

WORKSPACE_ID = "4188055811360976"  # Your workspace ID
WORKSPACE_HOSTNAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()

# Register workspace in BrickKit's global registry
registry = WorkspaceRegistry()
dev_workspace = registry.get_or_create(
    workspace_id=WORKSPACE_ID, name="free-edition-workspace", hostname=WORKSPACE_HOSTNAME, environment=Environment.DEV
)

print(f"Workspace: {dev_workspace.name}")
print(f"  ID: {dev_workspace.workspace_id}")
print(f"  Environment: {dev_workspace.environment.value}")

In [ ]:
# === DEFINE PRINCIPALS ===
# Service principals and groups that will own/access resources.
# These are environment-aware: names automatically get _dev/_acc/_prd suffixes.

# Service Principal for catalog ownership (convention requirement)
# Note: display_name defaults to resolved_name (e.g., spn_trading_platform_dev)
trading_platform_spn = ManagedServicePrincipal(name="spn_trading_platform")
trading_platform_spn.add_entitlement("workspace-access")
trading_platform_spn.add_entitlement("databricks-sql-access")

# Group for schema ownership and team access
# Note: display_name defaults to resolved_name (e.g., grp_quant_team_dev)
quant_team_group = ManagedGroup(name="grp_quant_team")
quant_team_group.add_entitlement("workspace-access")
quant_team_group.add_entitlement("databricks-sql-access")

# NOTE: Members can be added after creation using group_executor.sync_members()
# For this demo, we create the group without members.
# In production, add real users that exist in your workspace:
#   quant_team_group.add_user("real.user@yourcompany.com")
#   quant_team_group.add_service_principal("spn_trading_platform")

print(f"Service Principal: {trading_platform_spn.resolved_name}")
print(f"Group: {quant_team_group.resolved_name}")

In [ ]:
# === DEPLOY TEAM PRINCIPALS ===
# Create the service principal and group in Databricks before using them as owners.
# This is separate from the admin SPN (which is for privileged operations).
# The executors are idempotent - they return NO_OP if the principal already exists.

from brickkit.executors import GroupExecutor

group_executor = GroupExecutor(bootstrap_client, dry_run=DRY_RUN)

# Deploy trading platform SPN (will be catalog owner)
print(f"Deploying service principal: {trading_platform_spn.resolved_name}")
result = spn_executor.create(trading_platform_spn)
print(f"  {result.operation.value}: {result.message}")

# Deploy quant team group (will be schema owner)
print(f"Deploying group: {quant_team_group.resolved_name}")
result = group_executor.create(quant_team_group)
print(f"  {result.operation.value}: {result.message}")

In [ ]:
# === DEFINE TEAM ===
# Team brings together workspace, principals, and manages catalog bindings.

# Convert ManagedServicePrincipal/ManagedGroup to Principal for team membership
catalog_owner = Principal(
    name=trading_platform_spn.name,
    principal_type=PrincipalType.SERVICE_PRINCIPAL,
    add_environment_suffix=trading_platform_spn.add_environment_suffix,
)

schema_owner = Principal(
    name=quant_team_group.name,
    principal_type=PrincipalType.GROUP,
    add_environment_suffix=quant_team_group.add_environment_suffix,
)

# Create team and add workspace + principals
quant_team = Team(name="quant_trading")
quant_team.add_workspace(dev_workspace)
quant_team.add_principal(catalog_owner)
quant_team.add_principal(schema_owner)

print(f"Team: {quant_team.name}")
print(f"  Workspaces: {list(quant_team.workspaces.keys())}")
print(f"  Principals: {[p.resolved_name for p in quant_team.principals]}")

---
## 2. Sample Data

We'll use a small inline dataset of World Bank indicators. This lets you run the full demo quickly without external API calls.

In [ ]:
# === SAMPLE DATA ===
# 20 World Bank indicators with embedding text for vector search

SAMPLE_INDICATORS = [
    (
        "SP.POP.TOTL",
        "Population, total",
        "Total population counts all residents regardless of legal status or citizenship.",
        "Demographics",
    ),
    (
        "NY.GDP.MKTP.CD",
        "GDP (current US$)",
        "GDP at purchaser's prices is the sum of gross value added by all resident producers.",
        "Economy",
    ),
    (
        "NY.GDP.PCAP.CD",
        "GDP per capita (current US$)",
        "GDP per capita is gross domestic product divided by midyear population.",
        "Economy",
    ),
    (
        "SI.POV.DDAY",
        "Poverty headcount ratio at $2.15 a day",
        "Poverty headcount ratio at $2.15 a day is the percentage of the population living on less than $2.15 a day.",
        "Poverty",
    ),
    (
        "SI.POV.GINI",
        "Gini index",
        "Gini index measures the extent to which the distribution of income among individuals deviates from a perfectly equal distribution.",
        "Inequality",
    ),
    (
        "SL.UEM.TOTL.ZS",
        "Unemployment, total (% of labor force)",
        "Unemployment refers to the share of the labor force that is without work but available and seeking employment.",
        "Labor",
    ),
    (
        "FP.CPI.TOTL.ZG",
        "Inflation, consumer prices (annual %)",
        "Inflation as measured by the consumer price index reflects the annual percentage change in the cost of goods and services.",
        "Economy",
    ),
    (
        "SP.DYN.LE00.IN",
        "Life expectancy at birth, total (years)",
        "Life expectancy at birth indicates the number of years a newborn infant would live if patterns of mortality at birth were to stay the same.",
        "Health",
    ),
    (
        "SH.DYN.MORT",
        "Mortality rate, under-5 (per 1,000 live births)",
        "Under-five mortality rate is the probability per 1,000 that a newborn baby will die before reaching age five.",
        "Health",
    ),
    (
        "SE.ADT.LITR.ZS",
        "Literacy rate, adult total (% of people ages 15 and above)",
        "Adult literacy rate is the percentage of people ages 15 and above who can read and write a short simple statement.",
        "Education",
    ),
    (
        "SE.PRM.ENRR",
        "School enrollment, primary (% gross)",
        "Gross enrollment ratio is the ratio of total enrollment to the population of the age group that officially corresponds to the level of education.",
        "Education",
    ),
    (
        "EG.USE.ELEC.KH.PC",
        "Electric power consumption (kWh per capita)",
        "Electric power consumption measures the production of power plants and combined heat and power plants less transmission losses.",
        "Energy",
    ),
    (
        "EN.ATM.CO2E.PC",
        "CO2 emissions (metric tons per capita)",
        "Carbon dioxide emissions are those stemming from the burning of fossil fuels and the manufacture of cement.",
        "Environment",
    ),
    (
        "AG.LND.FRST.ZS",
        "Forest area (% of land area)",
        "Forest area is land under natural or planted stands of trees of at least 5 meters in situ.",
        "Environment",
    ),
    (
        "SH.XPD.CHEX.PC.CD",
        "Current health expenditure per capita (current US$)",
        "Current expenditures on health per capita in current US dollars.",
        "Health",
    ),
    (
        "IT.NET.USER.ZS",
        "Individuals using the Internet (% of population)",
        "Internet users are individuals who have used the Internet in the last 3 months.",
        "Technology",
    ),
    (
        "BX.KLT.DINV.CD.WD",
        "Foreign direct investment, net inflows (BoP, current US$)",
        "Foreign direct investment are the net inflows of investment to acquire a lasting management interest.",
        "Economy",
    ),
    (
        "GC.DOD.TOTL.GD.ZS",
        "Central government debt, total (% of GDP)",
        "Debt is the entire stock of direct government fixed-term contractual obligations to others outstanding.",
        "Economy",
    ),
    (
        "NE.EXP.GNFS.ZS",
        "Exports of goods and services (% of GDP)",
        "Exports of goods and services represent the value of all goods and other market services provided to the rest of the world.",
        "Trade",
    ),
    (
        "NE.IMP.GNFS.ZS",
        "Imports of goods and services (% of GDP)",
        "Imports of goods and services represent the value of all goods and other market services received from the rest of the world.",
        "Trade",
    ),
]

# Schema for the indicators table
INDICATORS_SCHEMA = StructType(
    [
        StructField("indicator_id", StringType(), False),
        StructField("indicator_name", StringType(), True),
        StructField("description", StringType(), True),
        StructField("topic", StringType(), True),
        StructField("embedding_text", StringType(), True),
    ]
)


def create_sample_dataframe(spark: SparkSession):
    """Create DataFrame from sample indicators with embedding text."""
    rows = [(ind_id, name, desc, topic, f"{name}. {desc}") for ind_id, name, desc, topic in SAMPLE_INDICATORS]
    return spark.createDataFrame(rows, INDICATORS_SCHEMA)


# Preview the sample data
sample_df = create_sample_dataframe(spark)
print(f"Sample data: {sample_df.count()} indicators")
sample_df.select("indicator_id", "indicator_name", "topic").show(5, truncate=40)

### (Optional) Fetch Real Data from World Bank API

Uncomment and run the cell below to fetch real indicator metadata. This takes several minutes.

In [ ]:
# === OPTIONAL: FETCH FROM WORLD BANK API ===
# Uncomment this cell to fetch real data (takes several minutes)

# %pip install wbgapi requests tqdm --quiet

# import wbgapi as wb
# import requests
# from requests.exceptions import RequestException, Timeout
# from tqdm import tqdm

# def fetch_worldbank_indicators(spark: SparkSession, limit: int = 100):
#     """Fetch indicator metadata from World Bank API."""
#     series_list = wb.series.info()
#     series_ids = [s.get("id") for s in series_list.items][:limit]
#
#     rows = []
#     for series_id in tqdm(series_ids, desc="Fetching"):
#         try:
#             url = f"https://api.worldbank.org/v2/indicator/{series_id}?format=json"
#             resp = requests.get(url, timeout=30)
#             resp.raise_for_status()
#             data = resp.json()
#             if len(data) >= 2 and data[1]:
#                 meta = data[1][0]
#                 name = meta.get("name", "") or ""
#                 desc = meta.get("sourceNote", "") or ""
#                 topics = meta.get("topics", []) or []
#                 topic = topics[0].get("value", "") if topics else ""
#                 embedding_text = f"{name}. {desc}".strip()
#                 rows.append((series_id, name, desc, topic, embedding_text))
#         except (RequestException, Timeout, ValueError) as e:
#             print(f"Skipping {series_id}: {e}")
#
#     return spark.createDataFrame(rows, INDICATORS_SCHEMA)

# # Fetch real data (uncomment to use)
# sample_df = fetch_worldbank_indicators(spark, limit=500)
# print(f"Fetched {sample_df.count()} indicators from World Bank API")

---
## 3. Define Governed Resources

Now we define our resources using BrickKit models. The convention automatically applies:
- Environment-specific naming (e.g., `quant_risk_dev`)
- Required governance tags
- Ownership rules validation

In [ ]:
# === ENVIRONMENT SETUP ===
# catalog_owner and schema_owner are already defined in the Team cell above

environment = ENV_MAP[ENVIRONMENT]

print(f"Catalog owner: {catalog_owner.resolved_name} ({catalog_owner.principal_type.value})")
print(f"Schema owner: {schema_owner.resolved_name} ({schema_owner.principal_type.value})")

In [ ]:
# === CATALOG ===
# NOTE: In Free Edition workspaces with "Default Storage", catalogs cannot be created via SDK.
# Create the catalog manually via UI first (with "Use default storage" checked), then reference it here.

catalog_name = convention.generate_name(SecurableType.CATALOG, environment)

catalog = Catalog(
    name=catalog_name,
    owner=catalog_owner,
    comment="Risk Analytics catalog for quantitative trading",
    isolation_mode=IsolationMode.ISOLATED,  # Default Storage catalogs are workspace-isolated
)

# Use Team to automatically configure workspace bindings
# This sets catalog.workspace_ids based on the team's workspace for this environment
quant_team.add_catalog(catalog)

# Apply convention (adds tags, validates rules)
convention.apply_to(catalog, environment)
errors = convention.get_validation_errors(catalog)
if errors:
    raise ValueError(f"Catalog validation failed: {errors}")

print(f"Catalog: {catalog.name}")
print(f"  Isolation Mode: {catalog.isolation_mode.value}")
print(f"  Workspace IDs: {catalog.workspace_ids} (auto-configured by Team)")
print(f"  Tags: {len(catalog.tags)}")

In [ ]:
# === SCHEMA ===
schema = Schema(
    name=SCHEMA_NAME,
    catalog_name=catalog.name,
    owner=schema_owner,
    comment="World Bank indicator metadata for vector search",
)

convention.apply_to(schema, environment)
errors = convention.get_validation_errors(schema)
if errors:
    raise ValueError(f"Schema validation failed: {errors}")

print(f"Schema: {schema.fqdn}")
print(f"  Tags: {len(schema.tags)}")

In [ ]:
# === TABLE ===
# Define the table structure with BrickKit (not just raw PySpark write)

table = Table(
    name=TABLE_NAME,
    catalog_name=catalog.name,
    schema_name=schema.name,
    owner=schema_owner,
    comment="World Bank indicator metadata with embeddings for semantic search",
    columns=[
        ColumnInfo(name="indicator_id", type="STRING", nullable=False, comment="World Bank indicator code"),
        ColumnInfo(name="indicator_name", type="STRING", nullable=True, comment="Human-readable indicator name"),
        ColumnInfo(name="description", type="STRING", nullable=True, comment="Full description of the indicator"),
        ColumnInfo(name="topic", type="STRING", nullable=True, comment="Category/topic of the indicator"),
        ColumnInfo(name="embedding_text", type="STRING", nullable=True, comment="Text used for embedding generation"),
    ],
    tags=[
        Tag(key="data_source", value="worldbank_api"),
        Tag(key="refresh_frequency", value="weekly"),
        Tag(key="contains_pii", value="false"),
    ],
)

convention.apply_to(table, environment)
errors = convention.get_validation_errors(table)
if errors:
    raise ValueError(f"Table validation failed: {errors}")

print(f"Table: {table.fqdn}")
print(f"  Columns: {len(table.columns)}")
print(f"  Tags: {len(table.tags)}")

In [ ]:
# === VECTOR SEARCH ENDPOINT ===
vs_endpoint = VectorSearchEndpoint(
    name=ENDPOINT_NAME,
    comment="Semantic search endpoint for risk analytics indicators",
    tags=[
        Tag(key="purpose", value="semantic_search"),
        Tag(key="model", value="databricks-bge-large-en"),
    ],
)

convention.apply_to(vs_endpoint, environment)
errors = convention.get_validation_errors(vs_endpoint)
if errors:
    raise ValueError(f"Endpoint validation failed: {errors}")

print(f"Endpoint: {vs_endpoint.resolved_name}")
print(f"  Tags: {len(vs_endpoint.tags)}")

In [ ]:
# === VECTOR SEARCH INDEX ===
# Use table.fqdn to reference the governed table

vs_index = VectorSearchIndex(
    name=INDEX_NAME,
    endpoint_name=ENDPOINT_NAME,
    source_table=table.fqdn,  # Reference the governed Table model
    primary_key="indicator_id",
    embedding_column="embedding_text",
    embedding_model="databricks-bge-large-en",
    pipeline_type="TRIGGERED",
    tags=[
        Tag(key="index_type", value="managed_embedding"),
    ],
)

convention.apply_to(vs_index, environment)
errors = convention.get_validation_errors(vs_index)
if errors:
    raise ValueError(f"Index validation failed: {errors}")

print(f"Index: {vs_index.resolved_name}")
print(f"  Source: {vs_index.source_table}")
print(f"  Endpoint: {vs_index.resolved_endpoint_name}")
print(f"  Tags: {len(vs_index.tags)}")

---
## 4. Deploy with BrickKit Executors

BrickKit executors handle:
- Idempotent create (skip if exists)
- Wait for provisioning
- Tag application
- **Permission grants** - Ensure teams have access after ownership change
- Error handling

In [ ]:
# === INITIALIZE CLIENTS AND EXECUTORS ===

# Default client for most operations (uses your notebook token)
ws_client = WorkspaceClient()
vs_client = VectorSearchClient()

# Grant admin SPN access to the catalog FIRST (using SQL via your user token)
# This is required before the admin SPN can manage the catalog
if CREDENTIALS_EXIST and not DRY_RUN:
    print(f"Granting admin SPN ({admin_spn.resolved_name}) access to catalog {catalog.resolved_name}...")
    
    # Use SQL grants - simpler and more reliable for bootstrap
    grant_sql = f"""
        GRANT MANAGE, USE CATALOG ON CATALOG `{catalog.resolved_name}` 
        TO `{ADMIN_SPN_APP_ID}`
    """
    try:
        spark.sql(grant_sql)
        print(f"  ✓ Granted MANAGE + USE CATALOG to {admin_spn.resolved_name}")
    except Exception as e:
        if "already has" in str(e).lower() or "already granted" in str(e).lower():
            print(f"  ✓ Privileges already granted")
        else:
            raise

# Create privileged client from stored credentials
PRIVILEGED_CLIENT = None
if CREDENTIALS_EXIST:
    PRIVILEGED_CLIENT = get_privileged_client(
        host=WORKSPACE_HOSTNAME,
        scope=SECRET_SCOPE,
        dbutils=dbutils,
    )
    print(f"✓ Privileged client ready (using {admin_spn.resolved_name})")

# Use privileged client for catalog operations, default for others
catalog_client = PRIVILEGED_CLIENT if PRIVILEGED_CLIENT else ws_client
if PRIVILEGED_CLIENT:
    print("Using privileged client (SPN) for catalog operations")
else:
    print("⚠ No privileged client - catalog updates may fail due to token restrictions")

# Initialize executors
catalog_executor = CatalogExecutor(catalog_client, dry_run=DRY_RUN)  # Uses privileged client
schema_executor = SchemaExecutor(ws_client, dry_run=DRY_RUN)
grant_executor = GrantExecutor(ws_client, dry_run=DRY_RUN)
endpoint_executor = VectorSearchEndpointExecutor(ws_client, dry_run=DRY_RUN)
index_executor = VectorSearchIndexExecutor(ws_client, dry_run=DRY_RUN)

print(f"Executors initialized (dry_run={DRY_RUN})")

In [ ]:
# === DEPLOY CATALOG ===
# In Free Edition with "Default Storage", create the catalog manually via UI first.
# The executor handles existing catalogs gracefully (returns NO_OP if already exists).
result = catalog_executor.create(catalog)
print(f"Catalog: {result.operation.value} - {result.message}")

In [ ]:
# === DEPLOY SCHEMA ===
result = schema_executor.create(schema)
print(f"Schema: {result.operation.value} - {result.message}")

In [ ]:
# === APPLY GRANTS ===
# After ownership change, the deploying user loses access. Grant permissions to the team group.
# This ensures the quant_team_group can access the catalog and work with the data.

# Grant the team group access to the catalog
# The group (grp_quant_team) needs USE_CATALOG + CREATE_SCHEMA to work in the catalog
catalog.grant(schema_owner, AccessPolicy.WRITER())

# Apply all accumulated grants to Databricks
results = grant_executor.apply_privileges(catalog.privileges)
for result in results:
    print(f"Grants on {result.resource_name}: {result.operation.value} - {result.message}")

# Show what was granted
print(f"\nPrivileges granted on catalog to {quant_team_group.resolved_name}:")
for priv in catalog.privileges:
    print(f"  - {priv.privilege.value}")

In [ ]:
# === WRITE DATA TO TABLE ===
# Using the governed Table model's fqdn

if not DRY_RUN:
    # Write with Delta format and Change Data Feed enabled
    (
        sample_df.write.format("delta")
        .option("delta.enableChangeDataFeed", "true")
        .mode("overwrite")
        .saveAsTable(table.fqdn)
    )

    # Verify
    count = spark.table(table.fqdn).count()
    print(f"Table: {table.fqdn} - {count} rows written")
else:
    print(f"[DRY RUN] Would write {sample_df.count()} rows to {table.fqdn}")

In [ ]:
# === DEPLOY VECTOR SEARCH ENDPOINT ===
result = endpoint_executor.create(vs_endpoint)
print(f"Endpoint: {result.operation.value} - {result.message}")

# Wait for endpoint to be online (uses executor's built-in wait logic)
if not DRY_RUN and result.operation.value == "CREATE":
    print("Waiting for endpoint to be online...")
    if endpoint_executor.wait_for_endpoint(vs_endpoint):
        print(f"Endpoint {vs_endpoint.resolved_name} is ONLINE")
    else:
        raise RuntimeError(f"Endpoint {vs_endpoint.resolved_name} failed to provision")

In [ ]:
# === DEPLOY VECTOR SEARCH INDEX ===
result = index_executor.create(vs_index)
print(f"Index: {result.operation.value} - {result.message}")

---
## 5. Test Vector Search

The index syncs asynchronously. Once ready, we can run similarity searches.

In [ ]:
# === CHECK INDEX STATUS ===

if not DRY_RUN:
    FULL_INDEX_NAME = f"{catalog.name}.{schema.name}.{vs_index.resolved_name}"

    index = vs_client.get_index(
        endpoint_name=vs_endpoint.resolved_name,
        index_name=FULL_INDEX_NAME,
    )
    status = index.describe().get("status", {})
    print(f"Index status: ready={status.get('ready', 'UNKNOWN')}")
    print(f"Message: {status.get('message', 'N/A')}")
else:
    print("[DRY RUN] Would check index status")

In [ ]:
# === RUN SIMILARITY SEARCH ===

if not DRY_RUN:
    TEST_QUERY = "poverty and inequality measures"

    try:
        results = index.similarity_search(
            query_text=TEST_QUERY,
            columns=["indicator_id", "indicator_name", "description", "topic"],
            num_results=5,
        )

        print(f"Search: '{TEST_QUERY}'")
        print("=" * 60)

        data = results.get("result", {}).get("data_array", [])
        for i, row in enumerate(data, 1):
            print(f"{i}. [{row[3]}] {row[1]}")
            print(f"   {row[2][:80]}...")
            print()

    except Exception as e:
        if "not ready" in str(e).lower() or "syncing" in str(e).lower():
            print("Index is still syncing. Please wait and try again.")
        else:
            raise
else:
    print("[DRY RUN] Would run similarity search")

---
## 6. What BrickKit Added (Governance Value)

Let's see what governance BrickKit applied automatically.

In [ ]:
# === GOVERNANCE SUMMARY ===


def display_resource_governance(name: str, resource):
    """Display governance metadata for a resource."""
    print(f"\n{'=' * 60}")
    print(f"{name}")
    print(f"{'=' * 60}")

    # Name (with environment suffix)
    if hasattr(resource, "resolved_name"):
        print(f"Name: {resource.resolved_name}")
    elif hasattr(resource, "fqdn"):
        try:
            print(f"Name: {resource.fqdn}")
        except ValueError:
            print(f"Name: {resource.name}")
    else:
        print(f"Name: {resource.name}")

    # Owner
    if hasattr(resource, "owner") and resource.owner:
        owner = resource.owner
        print(f"Owner: {owner.resolved_name} ({owner.principal_type.value})")

    # Isolation mode and workspace bindings (for catalogs)
    if hasattr(resource, "isolation_mode") and resource.isolation_mode:
        print(f"Isolation Mode: {resource.isolation_mode.value}")
    if hasattr(resource, "workspace_ids") and resource.workspace_ids:
        print(f"Workspace IDs: {resource.workspace_ids}")

    # Privileges (grants)
    if hasattr(resource, "privileges") and resource.privileges:
        print(f"Privileges ({len(resource.privileges)}):")
        for priv in resource.privileges:
            print(f"  - {priv.principal}: {priv.privilege.value}")

    # Request for Access (RFA)
    if hasattr(resource, "request_for_access") and resource.request_for_access:
        rfa = resource.request_for_access
        print(f"RFA Destination: {rfa.destination}")
        if rfa.instructions:
            print(f"RFA Instructions: {rfa.instructions}")

    # Columns (for tables)
    if hasattr(resource, "columns") and resource.columns:
        print(f"Columns: {len(resource.columns)}")

    # Tags
    if hasattr(resource, "tags") and resource.tags:
        print(f"Tags ({len(resource.tags)}):")
        for tag in sorted(resource.tags, key=lambda t: t.key):
            print(f"  - {tag.key}: {tag.value}")


# Display Team
print("=" * 60)
print("TEAM")
print("=" * 60)
print(f"  Name: {quant_team.name}")
print(f"  Workspaces:")
for env, ws in quant_team.workspaces.items():
    print(f"    - {env.value}: {ws.name} (ID: {ws.workspace_id})")
print(f"  Principals:")
for p in quant_team.principals:
    print(f"    - {p.resolved_name} ({p.principal_type.value})")

# Display defined principals (detailed)
print("\n" + "=" * 60)
print("PRINCIPALS (DETAILED)")
print("=" * 60)
print(f"\nService Principal: {trading_platform_spn.resolved_name}")
print(f"  Display Name: {trading_platform_spn.display_name}")
print(f"  Entitlements: {trading_platform_spn.entitlements}")

print(f"\nGroup: {quant_team_group.resolved_name}")
print(f"  Display Name: {quant_team_group.display_name}")
print(f"  Entitlements: {quant_team_group.entitlements}")
print(f"  Members ({len(quant_team_group.members)}):")
for member in quant_team_group.members:
    print(f"    - {member.resolved_name} ({member.principal_type.value})")

# Display governance for all resources
display_resource_governance("CATALOG", catalog)
display_resource_governance("SCHEMA", schema)
display_resource_governance("TABLE", table)
display_resource_governance("VECTOR SEARCH ENDPOINT", vs_endpoint)
display_resource_governance("VECTOR SEARCH INDEX", vs_index)

In [ ]:
# === CONVENTION RULES APPLIED ===

print("\n" + "=" * 60)
print("CONVENTION RULES")
print("=" * 60)
print(f"Convention: {convention.name} (v{convention.version})")
print()

for rule in convention.schema.rules:
    mode = "ENFORCED" if rule.mode.value == "enforced" else "ADVISORY"
    print(f"[{mode}] {rule.rule}")

print()
print("What this means:")
print("- Catalogs MUST be owned by service principals (not users)")
print("- All resources MUST be owned by SP or Group (no individual users)")
print("- Resources SHOULD have cost_center and team tags")
print("- BrickKit validated all these rules before deployment")

In [ ]:
# === WHAT YOU DIDN'T HAVE TO DO ===

print("\n" + "=" * 60)
print("WHAT BRICKKIT DID FOR YOU")
print("=" * 60)

benefits = [
    ("Team definition", f"Team '{quant_team.name}' brings together workspace + principals"),
    ("Principal definitions", f"Defined SP + Group with {len(quant_team_group.members)} members declaratively"),
    ("Workspace binding", f"team.add_catalog() auto-configured workspace IDs: {catalog.workspace_ids}"),
    ("Environment suffixes", f"All names automatically suffixed with '_{ENVIRONMENT}'"),
    ("Governance tags", f"{len(catalog.tags)} tags auto-applied from convention"),
    ("Ownership validation", "Verified catalog has SP owner, schema has Group owner"),
    ("Permission grants", f"Granted {len(catalog.privileges)} privileges to team group after ownership change"),
    ("Request for Access", "RFA configured with inheritance (table inherits from schema)"),
    ("Idempotent deployment", "Executors skip if resource exists, sync tags if needed"),
    ("Wait logic", "Built-in endpoint provisioning wait with timeout/retry"),
    ("Consistent patterns", "Same governance across Catalog, Schema, Endpoint, Index"),
]

for benefit, detail in benefits:
    print(f"\n{benefit}:")
    print(f"  {detail}")

print("\n" + "=" * 60)
print("Without BrickKit, you would manually:")
print("  - Define groups/SPNs with raw SDK calls")
print("  - Track which workspace IDs to bind to each catalog")
print("  - Add environment suffixes to every resource name")
print("  - Remember which tags to apply (and apply them consistently)")
print("  - Validate ownership rules before deployment")
print("  - Grant permissions to teams after changing ownership")
print("  - Configure RFA on each securable individually")
print("  - Write wait/retry logic for endpoint provisioning")
print("  - Handle idempotency (check exists, update tags, etc.)")
print("=" * 60)

---
## Summary

This demo showed:

1. **Convention Loading** - Governance rules from YAML
2. **Team Definition** - `Team` with `Workspace` and `Principal` members
3. **Principal Definition** - `ManagedServicePrincipal`, `ManagedGroup` with members
4. **Catalog Binding** - `team.add_catalog()` auto-configures workspace IDs
5. **Governed Models** - `Catalog`, `Schema`, `Table`, `VectorSearchEndpoint`, `VectorSearchIndex`
6. **Executors** - Idempotent deployment with built-in wait logic
7. **Permission Grants** - Ensure team access after ownership changes
8. **Automatic Governance** - Tags, naming, ownership validation

### BrickKit vs DAB Recap

- **DAB** handles: notebook sync, job definitions, workflow orchestration
- **BrickKit** handles: teams, principals, catalog/schema/table creation, workspace bindings, VS endpoint/index, grants, tags, validation
- **Together**: DAB runs this notebook as a job, BrickKit deploys the governed resources

### Next Steps

- Modify `conventions/financial_services.yml` to change governance rules
- Set `dry_run=false` to deploy for real
- Try different environments (`dev`, `acc`, `prd`) to see naming changes
- Add more workspaces to the team for multi-environment deployments
- Add your own data source instead of sample indicators